In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in c:\users\camilo\anaconda3\lib\site-packages (0.0)


In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [3]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [5]:
# Set features. This will also be used as your x values.
selected_features = df[['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co',
       'koi_fpflag_ec', 'koi_period', 'koi_time0bk', 'koi_impact',
       'koi_duration', 'koi_depth', 'koi_prad', 'koi_teq', 'koi_insol',
       'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff', 'koi_slogg',
       'koi_srad', 'ra', 'dec', 'koi_kepmag']]

In [6]:
y = df[["koi_disposition"]]

In [7]:
y = y.values.reshape(-1,)
y.shape

(6991,)

# Create a Train Test Split

Use `koi_disposition` for the y values

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(selected_features, y, test_size=0.20)

In [9]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_insol,koi_model_snr,koi_tce_plnt_num,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag
2478,0,0,0,0,13.981779,181.975114,0.444,2.95800,1018.4,2.09,553,22.10,95.6,1,4859,4.615,0.651,287.43918,43.832088,14.287
6838,1,0,0,0,331.926220,289.579200,0.026,10.84000,809.0,2.25,235,0.72,8.7,1,5652,4.577,0.798,287.04742,44.987968,15.986
2987,0,1,1,0,0.569961,131.802730,0.343,1.44600,44.2,1.04,2743,13463.04,18.9,1,5814,4.066,1.549,298.46341,48.321850,13.333
1314,0,1,0,0,116.521432,144.133780,1.260,5.57700,771.5,44.41,457,10.33,25.0,1,6056,4.179,1.411,286.37000,37.552521,13.806
4649,0,1,0,0,55.249007,164.978873,0.785,5.15147,205670.0,60.79,496,14.25,3452.5,1,6063,4.476,1.012,288.15308,43.473560,14.925


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [10]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler

X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [11]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

# Train the Model



In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()

model.add(Dense(units=100, activation='relu', input_dim=20))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

In [13]:
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [14]:
model.fit(X_train_scaled,
         y_train_categorical,
         epochs=100,
         shuffle=True,
         verbose=2)

Train on 5592 samples
Epoch 1/100
5592/5592 - 0s - loss: 0.5077 - accuracy: 0.7312
Epoch 2/100
5592/5592 - 0s - loss: 0.3835 - accuracy: 0.7904
Epoch 3/100
5592/5592 - 0s - loss: 0.3744 - accuracy: 0.7969
Epoch 4/100
5592/5592 - 0s - loss: 0.3682 - accuracy: 0.7979
Epoch 5/100
5592/5592 - 0s - loss: 0.3659 - accuracy: 0.7986
Epoch 6/100
5592/5592 - 0s - loss: 0.3632 - accuracy: 0.7985
Epoch 7/100
5592/5592 - 0s - loss: 0.3594 - accuracy: 0.8115
Epoch 8/100
5592/5592 - 0s - loss: 0.3553 - accuracy: 0.8115
Epoch 9/100
5592/5592 - 0s - loss: 0.3580 - accuracy: 0.8060
Epoch 10/100
5592/5592 - 0s - loss: 0.3528 - accuracy: 0.8155
Epoch 11/100
5592/5592 - 0s - loss: 0.3504 - accuracy: 0.8165
Epoch 12/100
5592/5592 - 0s - loss: 0.3465 - accuracy: 0.8228
Epoch 13/100
5592/5592 - 0s - loss: 0.3475 - accuracy: 0.8247
Epoch 14/100
5592/5592 - 0s - loss: 0.3475 - accuracy: 0.8185
Epoch 15/100
5592/5592 - 0s - loss: 0.3430 - accuracy: 0.8158
Epoch 16/100
5592/5592 - 0s - loss: 0.3417 - accuracy: 0.

In [15]:
print(f"Training Data Score: {model.evaluate(X_train_scaled, y_train_categorical)}")
print(f"Testing Data Score: {model.evaluate(X_test_scaled, y_test_categorical)}")

5592/5592 [==============================] - 0s 36us/sample - loss: 0.2128 - accuracy: 0.9070
Training Data Score: [0.2128437742995943, 0.90701]
1399/1399 [==============================] - 0s 30us/sample - loss: 0.3324 - accuracy: 0.8849
Testing Data Score: [0.3324328494165692, 0.8849178]


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [ ]:
list = [25, 50, 75, 100]

for x in list:
    for i in list:
        model2 = Sequential()

        model2.add(Dense(units=x, activation='relu', input_dim=20))
        model2.add(Dense(units=i, activation='relu'))
        model2.add(Dense(units=3, activation='softmax'))
        
        model2.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])
        
        model2.fit(X_train_scaled,
         y_train_categorical,
         epochs=100,
         shuffle=True,
         verbose=0)
        
        print(f"Units: {x} and {i}, Testing Score: {model2.evaluate(X_test_scaled, y_test_categorical)}")
        

1399/1399 [==============================] - 0s 69us/sample - loss: 0.3348 - accuracy: 0.8420
Units: 25 and 25, Testing Score: [0.3347735819516649, 0.84203005]
1399/1399 [==============================] - 0s 69us/sample - loss: 0.3443 - accuracy: 0.8477
Units: 25 and 50, Testing Score: [0.34433667433099974, 0.8477484]
1399/1399 [==============================] - 0s 66us/sample - loss: 0.3528 - accuracy: 0.8284
Units: 25 and 75, Testing Score: [0.35283599381705877, 0.8284489]
1399/1399 [==============================] - 0s 122us/sample - loss: 0.3166 - accuracy: 0.8642
Units: 25 and 100, Testing Score: [0.3166173193776838, 0.86418873]
1399/1399 [==============================] - 0s 117us/sample - loss: 0.3377 - accuracy: 0.8427
Units: 50 and 25, Testing Score: [0.33769288437548833, 0.8427448]
1399/1399 [==============================] - 0s 207us/sample - loss: 0.3265 - accuracy: 0.8556
Units: 50 and 50, Testing Score: [0.32648668359909166, 0.85561115]
1399/1399 [========================

# Save the Model

In [ ]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'your_name.sav'
joblib.dump(your_model, filename)